#### Torch 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

class IrisNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.fc_middle = nn.Linear(16, 16)
        self.bn2 = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc_middle(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

data = load_iris()
X = torch.tensor(data.data, dtype=torch.float32)
y = torch.tensor(data.target, dtype=torch.long)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = IrisNet()
writer = SummaryWriter(log_dir='logs/logs_torch')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(40):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    writer.add_scalar('train_loss', loss.item(), epoch)
    train_acc = (torch.argmax(outputs, dim=1) == y_train).float().mean()
    writer.add_scalar('train_acc', train_acc.item(), epoch)
    loss.backward()
    optimizer.step()

writer.close()
pred = torch.argmax(model(X_test), dim=1)
acc = (pred == y_test).float().mean()
print("Test accuracy:", acc.item())


Test accuracy: 0.8666666746139526


#### Tensorflow

In [2]:
%%time
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
model_tf = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(3, activation='softmax')
])
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
tb_callback_tf = TensorBoard(log_dir='logs/logs_tf', histogram_freq=1, write_graph=False, write_images=False)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True)
model_tf.fit(
    X_train, y_train, epochs=50, callbacks=[tb_callback_tf, early_stopping],
    validation_data=(X_test, y_test))


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 211ms/step - accuracy: 0.2837 - loss: 1.6201 - val_accuracy: 0.3333 - val_loss: 1.7198
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.2783 - loss: 1.4528 - val_accuracy: 0.3333 - val_loss: 1.6475
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.2869 - loss: 1.5303 - val_accuracy: 0.3333 - val_loss: 1.5995
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3835 - loss: 1.2591 - val_accuracy: 0.3333 - val_loss: 1.5454
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.3723 - loss: 1.2489 - val_accuracy: 0.3333 - val_loss: 1.4967
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5148 - loss: 1.0767 - val_accuracy: 0.3333 - val_loss: 1.4804
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5660 - loss: 1.0312 - val_accuracy: 0.3333 - val_loss: 1.4573
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.4858 - loss: 1.1346 - val_accuracy: 0.2333 - val_loss: 1.4295

#### Pytorch-lightning

In [3]:
%%time
import pytorch_lightning as pl
from torch.utils.data import TensorDataset, DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import MulticlassAccuracy
class IrisLightningModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = IrisNet()
        self.criterion = nn.CrossEntropyLoss()
        self.acc = MulticlassAccuracy(num_classes=3)
    def training_step(self, batch, batch_idx):
        x, y = batch
        outputs = self.model(x)
        loss = self.criterion(outputs, y)
        acc = self.acc(outputs, y)
        self.log('train_loss', loss)
        self.log('train_acc', acc, on_epoch=True)
        return loss
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.001)
dataset_train = TensorDataset(X_train, y_train)
train_loader = DataLoader(dataset_train, batch_size=16, shuffle=True)
logger = TensorBoardLogger('logs/logs_lightning', name='iris')
trainer = pl.Trainer(max_epochs=30, logger=logger)
model_lightning = IrisLightningModule()
trainer.fit(model_lightning, train_loader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | IrisNet            | 467    | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | acc       | MulticlassAccuracy | 0      | train
---------------------------------------------------------
467       Trainable params
0         Non-trainable params
467       Total params
0.002     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
c:\Users\cah\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
c:\Users\cah\AppData\Local\Programs\Pyt

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


CPU times: total: 4.94 s
Wall time: 8.5 s


In [4]:
%load_ext tensorboard

%tensorboard --logdir logs --port=6006


Reusing TensorBoard on port 6006 (pid 2360), started 21:10:18 ago. (Use '!kill 2360' to kill it.)